In [1]:
import re
import pandas as pd
from tf.app import use

In [2]:
A = use('bhsa', hoist=globals())

This is Text-Fabric 9.1.6
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

122 features found and 0 ignored


In [3]:
whole_text = " ".join([T.text(word, fmt='text-phono-full') for word in F.otype.s('word')])

verses = whole_text.split(".")

In [4]:
def clean_text(sentence, replacement_vals):
    for k, v in replacement_vals.items():
        sentence = re.sub(k, v, sentence)
    return sentence.strip()

In [5]:
#to bring Text Fabric's 'text-phono-full' format more in line with the SBL transcription system

replacement_vals = {" f " : "", 
                    " s " : "", 
                    "ˌ" : "", 
                    "ˈ" : "", 
                    "\." : "", 
                    "-" : " ", 
                    "\[" : "", 
                    "\]" : "", 
                    "v" : "b", 
                    "ḡ" : "g", 
                    "ḏ" : "d", 
                    "ḵ" : "k", 
                    "f" : "p", 
                    "ṯ" : "t",  
                    "ᵊ" : "ə", 
                    "ᵉ" : "ĕ", 
                    "ₐ" : "a", 
                    "ᵃ" : "ă",
                    "eʸ" : "ê",
                    "āʸ" : "āy",
                    "eh" : "ê",
                    "\*" : "",
                    r'\b(\w)\1*' : r'\1'}

In [6]:
df = pd.DataFrame({'Verse' : ['{0} {1}:{2}'.format(T.sectionFromNode(verse)[0], T.sectionFromNode(verse)[1], T.sectionFromNode(verse)[2]) for verse in F.otype.s("verse")],
          'Text' : [clean_text(verse, replacement_vals) for verse in verses]})

df.head()

,Verse,Text
0,Genesis 1:1,bə rēšît bārā ʔĕlōhîm ʔēt ha šāmayim wə ʔ...
1,Genesis 1:2,wə hā ʔāreṣ hāyətā tōhû wā bōhû wə ḥōšek ...
2,Genesis 1:3,wa yōmer ʔĕlōhîm yəhî ʔôr wa yəhî ʔôr
3,Genesis 1:4,wa yar ʔĕlōhîm ʔet hā ʔôr kî ṭôb wa yabd...
4,Genesis 1:5,wa yiqrā ʔĕlōhîm lā ʔôr yôm wə la ḥōšek ...


In [7]:
#Relace the full names of biblical books with their SBL handbook of style abbreviations, p. 124

replacement_vals_books = {'Genesis' : 'Gen', 
                          'Exodus' : 'Exod', 
                          'Numbers' : 'Num', 
                          'Leviticus' : 'Lev', 
                          'Deuteronomy' : 'Deut', 
                          'Joshua' : 'Josh', 
                          '1_Samuel' : '1 Sam', 
                          '2_Samuel' : '2 Sam', 
                          '1_Kings' : '1 Kgs', 
                          '2_Kings' : '2 Kgs', 
                          'Isaiah' : 'Isa', 
                          'Jeremiah' : 'Jer', 
                          'Ezekiel' : 'Ezek', 
                          'Hosea' : 'Hos', 
                          'Obadiah' : 'Obad', 
                          'Michah' : 'Mic', 
                          'Nahum' : 'Nah', 
                          'Habakkuk' : 'Hab', 
                          'Zephaniah' : 'Zeph', 
                          'Haggai' : 'Hag', 
                          'Zechariah' : 'Zech', 
                          'Malachi' : 'Mal', 
                          'Psalms' : 'Ps', 
                          'Proverbs' : 'Prov', 
                          'Song_of_Songs' : 'Song', 
                          'Ecclesiastes' : 'Eccl', 
                          'Lamentations' : 'Lam', 
                          'Esther' : 'Esth', 
                          'Daniel' : 'Dan', 
                          'Nehemiah' : 'Neh', 
                          '1_Chronicles' : '1 Chr', 
                          '2_Chronicles' : '2 Chr'}

for k, v in replacement_vals_books.items():
    df['Verse'].replace(k, v, inplace=True, regex=True)

In [8]:
#Text Fabric does not include a verse marker between 1 Kings 16:26 and 27
first_half = df.iloc[0:9227]
second_half = df.iloc[9227:]
second_half['Text'] = second_half['Text'].shift(1)

/var/folders/f9/058hws5n0db855sskzpm2bh80000gq/T/ipykernel_84381/2328280735.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_half['Text'] = second_half['Text'].shift(1)


In [9]:
first_half.iloc[9226]['Text'] = 'wa yēlek  bə kol  derek  yārobʕām  ben  nəbāṭ  û bə ḥaṭṭātô  ʔăšer  heḥĕṭî  ʔet  yiśrāʔēl  lə hakʕîs  ʔet  yəhwāh  ʔĕlōhê  yiśrāʔēl  bə hablêhem'
second_half.iloc[0]['Text'] = 'wə yeter  dibrê  ʕomrî  ʔăšer  ʕāśā  û gəbûrātô  ʔăšer  ʕāśā  hă lō  hēm  kətûbîm  ʕal  sēper  dibrê  ha yāmîm  lə maləkê  yiśrāʔēl'
second_half.dropna(inplace=True)
df = pd.concat([first_half, second_half])

/var/folders/f9/058hws5n0db855sskzpm2bh80000gq/T/ipykernel_84381/4268817676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_half.dropna(inplace=True)


In [10]:
df.set_index('Verse', inplace=True)

In [12]:
df.tail()

,Text
Verse,
2 Chr 36:19,wa yiśrəpû ʔet bêt hā ʔĕlōhîm wa yənattəṣû...
2 Chr 36:20,wa yegel ha šəʔērît min ha ḥereb ʔel bābe...
2 Chr 36:21,lə mallôt dəbar yəhwāh bə pî yirməyāhû ʕa...
2 Chr 36:22,û bi šənat ʔaḥat lə kôreš melek pāras li ...
2 Chr 36:23,kō ʔāmar kôreš melek pāras kol mamləkôt ...


In [13]:
#remove Aramaic texts
df.drop('Jer 10:11', inplace=True)
df.drop(df.loc['Dan 2:5' : 'Dan 7:28'].index, inplace=True)
df.drop(df.loc['Ezra 4:8' : 'Ezra 6:18'].index, inplace=True)
df.drop(df.loc['Ezra 7:12' : 'Ezra 7:26'].index, inplace=True)

In [14]:
df.loc['Dan 2:4']['Text'] = 'wa yədabbərû  ha kaśdîm  la  melek  ʔărāmît'

df.loc['Dan 2:4']['Text']

'wa yədabbərû  ha kaśdîm  la  melek  ʔărāmît'

In [15]:
df.to_csv('HebrewBiblebyVerse.csv')

In [ ]:
def retrieve_text(string):
    verses = string.split("-")
    return verses, verse2

In [ ]:
":".join(re.sub(r'(\d?\s?\w+\s\d+:)(\d+-)(\d+)', r'\1\2\1\3', '1 Sam 1:1-3').split("-"))

In [ ]:
re.sub(r'(\d?\s?\w+\s\d+:)(\d+,)(\d+)', r'\1\2\1\3', '1 Sam 1:1,3').split(",")

In [ ]:
re.sub(r'(\d?\s?\w+\s)(\d+:\d+;\s)(\d+:\d+)', r'\1\2\1\3', '1 Sam 1:1; 3:5').split(";")

In [ ]:
df.loc['Dan 2:4' : 'Dan 7:28']

In [ ]:
df.loc['Gen 27:20'][-1]